In [ ]:
import pandas as pd
import re
import os
import glob
import concurrent.futures
from tqdm import tqdm
import time
import numpy as np
import concurrent.futures

# COMPANY DATA METHOD NEW

In [ ]:
# create column names and dataframe

path = 'sec_edgar_filings/917520/10-K/0000917520-20-000010.txt'
file = open(path)
lines = [line for line in file.readlines() if line.strip()]
lines = [x.strip() for x in lines]
for i in range(len(lines)):
        try:
            lines[i] = re.sub('\t+','',lines[i])
        except:
            pass
file.close()

header_row = lines.index('</SEC-HEADER>')
companystart = lines.index('FILER:') + 2
lines = lines[companystart:header_row]
col = []
for i in lines:
    col.append(i.split(':')[0])
col.insert(13,'STREET 2')
col.insert(20,'STREET 2')
for ii in range(9):
    col.append(col[24] + ' ' + f'{ii + 2}')
    col.append(col[25] + ' ' + f'{ii + 2}')
    col.append(col[26] + ' ' + f'{ii + 2}')

df = pd.DataFrame(index = col)

In [ ]:
# create function that find the first/second occurence of a specified char in string

def find_2nd(string, substring):
    return string.find(substring, string.find(substring) + 1)
def find_1st(string, substring):
    return string.find(substring, string.find(substring))

In [ ]:
# integrate paths and modify it to be linux/os/windows compatible

paths10k = glob.glob('sec_edgar_filings/*/10-K/*.txt')
paths8k = glob.glob('sec_edgar_filings/*/8-K/*.txt')
pathsSC = glob.glob('sec_edgar_filings/*/SC 13G/*.txt')

paths = paths10k + paths8k + pathsSC
for i in range(len(paths)):
    try:
        paths[i] = paths[i].replace('\\','/')
    except:
        pass

In [ ]:
# only select one path from each company, can be from SG 13C or 8-K or 10-K

path = []
for i in range(len(paths)):
    det = True
    for ii in path:
        if paths[i][find_1st(paths[i],'/')+1 : find_2nd(paths[i],'/')] in ii:
            det = False
    if det:
        path.append(paths[i])

In [ ]:
# integrate all to be one big df

not_in_format = []
for p in tqdm(path):
    try:
        file = open(p)
        lines = [line for line in file.readlines() if line.strip()]
        lines = [x.strip() for x in lines]
        for i in range(len(lines)):
                try:
                    lines[i] = re.sub('\t+','',lines[i])
                except:
                    pass
        file.close()

        try:
            FB_row = lines.index('FILED BY:')
        except:
            FB_row = 9999
        header_row = lines.index('</SEC-HEADER>')
        if FB_row > header_row:
            end = header_row
        else:
            end = FB_row
        companystart = lines.index('COMPANY DATA:') + 1
        lines = lines[companystart:end]
        for i in range(len(col[:22])):
            if lines[i].split(':')[0] != col[i]:
                lines.insert(i,col[i] + ':')
        
        
        values = []
        for i in lines:
            values.append(i.split(':')[1])
        if len(values) != df.shape[0]:
            for i in range(df.shape[0]-len(values)):
                values.append('None')
        if lines.index('MAIL ADDRESS:') != 18:
            not_in_format.append(values[1])
            continue
        df[f'{values[1]}'] = values
    except:
        continue

In [ ]:
df.T.to_csv('companydata.csv')


In [ ]:
len(not_in_format)

In [ ]:
df

# FILING DATA

In [ ]:
def get_file(path):
    infofile = []
    infoFB = []
#     tmp = []
    for i in path:
        x = i.split('/')[1]
        
        if 'SC 13G' not in i:
            with open(i, 'r') as f:
                info = f.readlines()
            info = [x.rstrip('\n') for x in info]
            info = [x.strip('\t') for x in info]
            fileend = info.index('FILER:')
            tmp = info[:fileend]
            tmp = ['Filing_Company_CIK:' + x] + tmp
        else:
            with open(i, 'r') as f:
                infoSC = f.readlines()
            infoSC = [x.rstrip('\n') for x in infoSC]
            infoSC = [x.strip('\t') for x in infoSC]
            
            fileend = infoSC.index('SUBJECT COMPANY:') - 1
            tmp = infoSC[:fileend]
            tmp = ['Filing_Company_CIK:' + x] + tmp
            
            FBstart = infoSC.index('FILED BY:')
            header_row = infoSC.index('</SEC-HEADER>')
            infoFB = infoSC[FBstart:header_row]
#             infoFB = ['Primary_CIK:' + x] + infoFB
            
        infofile.append(tmp)
    
    return (infofile, infoFB)

In [ ]:
def cleaning_trans_comp(info):

# remove NAs and tabs
    info = [x for x in info if x == x]

    for i in range(len(info)):
        try:
            info[i] = re.sub('\t+','',info[i])
        except:
            pass
    
# distinguish duplicated keys in different files
    for i in range(len(info)):
        for ii in range(i+1,len(info)):
            if (info[i].split(':')[0] + ':') in info[ii] and info[i] != '':
                info[ii] = info[ii].split(':')[0] + '_' + str( "%04d" % ii) + ':' + info[ii].split(':')[1]

# transform to dictionary
    dic = {}
    for i in info:
        try:
            dic.update({i.split(':')[0]:i.split(':')[1].strip('\t')})
        except:
            pass
        
# transform to the dataframe
    df = pd.DataFrame(dic, index = [0])
    
# put the address together(street, city, state, zip)
    for i, col in enumerate(df.columns):
        if 'ADDRESS' in col:
            df[col] = df[col].str.cat(df.iloc[:,i+1:i+6],sep = ',')
            df.iloc[:, i+1:i+6] = ''

    return df

In [ ]:
def cleaning_file(info):
# remove NAs and tabs
    info = [x for x in info if x == x]

    for i in range(len(info)):
        try:
            info[i] = info[i].replace('\t','')
        except:
            pass
    
    for i in range(len(info)):
        info[i] = info[i].lstrip()
        if '>' in info[i] and ':' not in info[i]:
            info[i] = info[i].replace('>', '>:')
        elif '>' in info[i] and ':' in info[i]:
            info[i] = info[i].replace(':',';')
            info[i] = info[i].replace('>', '>:')
              
    
# distinguish duplicated keys in different files
    for i in range(len(info)):
        for ii in range(i+1,len(info)):
            if (info[i].split(':')[0] + ':') in info[ii] and info[i] != '':
                info[ii] = info[ii].split(':')[0] + '_' + str( "%04d" % ii) + ':' + info[ii].split(':')[1]
                
                
    info = [x for x in info if x != '']

    return info

In [ ]:
def trans(info):
# transform to dataframe
    df_file = pd.DataFrame()
    
    dic_file = {}
    idx = 0
    for k in info:
        for j in k:
            try:
                if j.split(':')[1] != '':
                    dic_file.update({j.split(':')[0]:j.split(':')[1]})
            except:
                pass
        tmp = pd.DataFrame(dic_file, index = [idx])

        df_file = df_file.join(tmp.T,how = 'outer')

        dic_file = {}
        idx += 1
    
    return df_file

In [ ]:
companies = glob.glob('sec_edgar_filings/*')
len(companies)

In [ ]:
companies = glob.glob('sec_edgar_filings/*/*/*.txt')
companies[14369:14371]

In [ ]:
datafile = pd.DataFrame()
companies = glob.glob('sec_edgar_filings/*')
jump = 0

# def final_function(companies):
for company in tqdm(companies):
    try:
        path = glob.glob(company + '/*/*.txt')

        infofile, infoFB = get_file(path)

        infofile = list(map(cleaning_file, infofile))
        infofile = trans(infofile)


        if infoFB!= []:
            infoFB = cleaning_trans_comp(infoFB)

            new = []
            for i in infoFB.columns:
                new.append('FILED BY:' + i)
            infoFB.columns = new

            infoFB.index = infofile.T[infofile.T['CONFORMED SUBMISSION TYPE'] == 'SC 13G'].index
            infofile = infofile.T.join(infoFB, how = 'left', lsuffix = 'CONFORMED SUBMISSION TYPE', rsuffix = 'FORM TYPE')

            result = infofile.T.drop_duplicates()
            df = result.T
        else:
            df = infofile.T.drop_duplicates()

        df.index = df.index.map(lambda x: x + jump)
        jump += len(df.T.columns)
        datafile = datafile.join(df.T, how = 'outer')
    except:
        pass
#     return datafile

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     executor.map(final_function,companies)
datafile.to_csv('temp_filedata.csv', index = False)

In [ ]:
# reorder the columns    
cols = list(datafile.T.columns)

news = []
olds = []
for i in cols:
    if 'FILED BY:' in i:
        news.append(i)
    else:
        olds.append(i)
cols = olds + news
cols.insert(0,cols.pop(cols.index('CONFORMED SUBMISSION TYPE')))
cols.insert(0,cols.pop(cols.index('Filing_Company_CIK')))
cols.insert(cols.index('FILED BY:BUSINESS ADDRESS') ,cols.pop(cols.index('FILED BY:COMPANY CONFORMED NAME')))


final_file = datafile.T.loc[:,cols].drop(['FILED BY:FILED BY'], axis = 1)

In [ ]:
final_file.to_csv('filedata.csv', index = False)

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
final_file.head()

# END --------------------------------------------------------

# COMPANY DATA METHOD ORIGINAL


In [ ]:
def get_comp(path):
    infocomp = []
    for i in path:
        if 'SC 13G' not in i:
            df = pd.read_fwf(i,header = None)
            info = df[0].to_list()
            header_row = info.index('</SEC-HEADER>')
            companystart = info.index('FILER:') + 3
            infocomp += info[companystart:header_row]
        else:
            df = pd.read_fwf(i,header = None)
            infoSC = df[0].to_list() 
            companystart = infoSC.index('SUBJECT COMPANY:') + 3
            companyend = infoSC.index('FILED BY:') - 1
            infocomp += infoSC[companystart:companyend]
            
    return infocomp
    

In [ ]:
def cleaning_comp(info):

# remove NAs and tabs
    info = [x for x in info if x == x]

    for i in range(len(info)):
        try:
            info[i] = re.sub('\t+','',info[i])
        except:
            pass
    
# distinguish duplicated keys in different files
    for i in range(len(info)):
        for ii in range(i+1,len(info)):
            if (info[i].split(':')[0] + ':') in info[ii]:
                info[ii] = info[ii].split(':')[0] + '_' + str( "%04d" % ii) + ':' + info[ii].split(':')[1]

# for each file's data, adding filetype symbol
    temp = ''
    for i in ['FILED AS OF DATE','DATE AS OF CHANGE','CONFORMED PERIOD OF REPORT','ACCESSION NUMBER']:
        for ii in range(len(info)):
            if i in info[ii]:
                for iii in range(ii-8,ii+2):
                    if 'CONFORMED SUBMISSION TYPE' in info[iii]:
                        temp = info[iii].split(':')[1]
                info[ii] = temp + '_' + info[ii]
            
# transform to dictionary
    dic = {}
    for i in info:
        try:
            dic.update({i.split(':')[0]:i.split(':')[1]})
        except:
            pass
        
# transform to the dataframe
    df = pd.DataFrame(dic, index = [dic['CENTRAL INDEX KEY']])
    
# put the address together(street, city, state, zip)
    for i, col in enumerate(df.columns):
        if 'ADDRESS' in col:
            df[col] = df[col].str.cat(df.iloc[:,i+1:i+6],sep = ',')
            df.iloc[:, i+1:i+6] = ''
        
    return df

In [ ]:
def dropduplicates_comp(infocomp):   
    information = cleaning_comp(infocomp).T.drop_duplicates()   
    return information

In [ ]:
datacomp = pd.DataFrame()
companies = glob.glob('sec_edgar_filings/*')
for company in tqdm(companies):
    path = glob.glob(company + '/*/*.txt')
    infocomp = get_comp(path)
    infocomp = dropduplicates_comp(infocomp)
    datacomp = datacomp.join(infocomp, how = 'outer')
final_comp = datacomp.T

In [ ]:
colnames = final_comp.columns.to_list()
done = []
for i in range(len(colnames)):
    counter = 1
    if colnames[i].split('_')[0] not in done:
        colnames[i] = colnames[i].split('_')[0]
        for ii in range(len(colnames)):
            if colnames[i] in colnames[ii]:
                colnames[ii] = colnames[ii].split('_')[0] + '_' + str(counter)
                counter += 1
        done.append(colnames[i])
final_comp.columns = colnames

In [ ]:
final_comp.fillna('Null',inplace = True)
columns = final_comp.columns.to_list()
iterator = 2
for i in range(final_comp.shape[0]):    
    while (iterator < len(columns)-1):
        if ((columns[iterator].split('_')[0] == columns[iterator-1].split('_')[0]) and (final_comp.iloc[i,iterator-1] == 'Null')):
            final_comp.iloc[i,iterator-1] = final_comp.iloc[i,iterator]
            final_comp.iloc[i,iterator] = 'Null'
        iterator = iterator + final_comp.shape[0]
            
            

In [ ]:
pd.options.display.max_columns = 999
final_comp